# ETF Monthly Scanner Notebook

This notebook wraps `etf.py` to build the universe from `etfdb_screener.csv`, fetch monthly data via yfinance, compute metrics, and produce final CSVs for AI analysis.


In [1]:
# Dependency check (optional)
import sys, subprocess, sys

def ensure(pkgs):
    for p in pkgs:
        try:
            __import__(p)
        except ImportError:
            print(f"Installing {p}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", p])

ensure(["pandas", "numpy", "yfinance", "matplotlib", "requests_cache"])


In [2]:
# Parameters
CSV_PATH = "/Users/vnktajay/Desktop/Etfs/etfdb_screener.csv"  # or None to auto-detect
RISK_FREE = 0.045
MIN_AUM_M = 100


In [3]:
# Imports and reload
import importlib, os, sys
sys.path.append("/Users/vnktajay/Desktop/Etfs")
import etf
importlib.reload(etf)

csv_resolved = etf.resolve_csv_path(CSV_PATH)
universe = etf.build_universe(csv_resolved)
universe_size = len(universe)
universe_size


4024

In [4]:
# Run analysis
M, corr = etf.compute_all(universe, RISK_FREE, MIN_AUM_M, csv_resolved)
R = etf.composite_rank(M)
L, P, LV = etf.shortlists(M)
B = etf.shortlist_bonds(M)
print(len(M), "metrics rows")
print("files:", [f for f in os.listdir("/Users/vnktajay/Desktop/Etfs") if f.endswith('.csv')])


2005 metrics rows
files: ['shortlist_pullbacks_above_10mo.csv', 'shortlist_bonds_high_yield_low_vol.csv', 'etf_fundamentals.csv', 'etf_ranked.csv', 'shortlist_momo_leaders.csv', 'etf_monthly_metrics.csv', 'shortlist_lowvol_defensive.csv', 'etfdb_screener.csv', 'etf_final_dataset.csv', 'etf_pairwise_corr.csv', 'etf_universe_used.csv']


In [5]:
# Inspect outputs
import pandas as pd
base = "/Users/vnktajay/Desktop/Etfs"
ranked = pd.read_csv(os.path.join(base, "etf_ranked.csv"))
final = pd.read_csv(os.path.join(base, "etf_final_dataset.csv"))
leaders = pd.read_csv(os.path.join(base, "shortlist_momo_leaders.csv"))
pullbacks = pd.read_csv(os.path.join(base, "shortlist_pullbacks_above_10mo.csv"))

ranked.head(12)


,symbol,name,category,last_m_close,ret_1m_%,ret_3m_%,ret_6m_%,ret_12m_%,mom_12_1_%,CAGR_5y_%,...,ADX14_monthly,Above_10MO,%_Above_10MO,Expense_%,AUM_M,Div_Yield_%,BidAsk_Proxy_%,AvgDollarVol_60d,Corr_vs_SPY,score
0,NVDY,YieldMax NVDA Option Income Strategy ETF,Equity,16.430000,3.548840,13.690128,39.343140,38.168363,33.433037,60.364964,...,33.139824,1,19.062040,1.27,1789.110,NaN,0.1,6.462147e+07,NaN,1.715852
1,CNEQ,Alger Concentrated Equity ETF,Equity,33.490002,4.558236,12.760946,39.343140,45.168009,33.821287,40.332696,...,NaN,1,19.062040,0.55,116.658,NaN,0.1,1.902132e+06,NaN,1.704091
2,BTC,Grayscale Bitcoin Mini Trust ETF,Currency,49.320000,2.900061,3.287957,35.160311,75.204264,33.821287,73.112460,...,NaN,1,10.731925,0.15,5225.670,NaN,0.1,5.753494e+07,NaN,1.677761
3,NVDL,GraniteShares 2x Long NVDA Daily ETF,Equity,79.519997,-4.296546,12.922463,39.343140,36.655780,33.821287,189.592695,...,30.614098,1,19.062040,1.06,3933.910,NaN,0.1,9.851586e+08,NaN,1.676601
4,NUKZ,Range Nuclear Renaissance Index ETF,Equity,58.799999,0.017007,3.960392,39.343140,59.175897,33.821287,56.999910,...,NaN,1,18.633588,0.85,503.792,NaN,0.1,1.109750e+07,NaN,1.646316
5,FBL,GraniteShares 2x Long META Daily ETF,Equity,47.500000,7.054319,3.825137,39.343140,45.304381,33.821287,130.137079,...,23.178263,1,19.062040,1.15,155.108,NaN,0.1,2.464100e+07,NaN,1.646199
6,SHLD,Global X Defense Tech ETF,Equity,62.439999,0.289111,3.839235,34.399868,68.659411,33.821287,57.032603,...,NaN,1,19.062040,0.50,3467.520,NaN,0.1,6.306615e+07,NaN,1.640710
7,NVDX,T-Rex 2X Long NVIDIA Daily Target ETF,Equity,16.280001,-4.403993,11.698119,39.343140,31.707974,33.821287,158.962101,...,NaN,1,19.062040,1.05,611.561,NaN,0.1,2.207113e+08,NaN,1.636100
8,ALAI,Alger AI Enablers & Adopters ETF,Equity,35.580002,5.437851,15.332257,39.343140,54.011966,33.821287,47.950687,...,NaN,1,19.062040,0.55,158.663,NaN,0.1,2.721977e+06,NaN,1.634788
9,NVDU,Direxion Daily NVDA Bull 2X Shares,Equity,111.739998,-4.422210,13.706579,39.343140,39.171614,33.821287,149.001241,...,NaN,1,19.062040,0.97,623.794,NaN,0.1,7.956480e+07,NaN,1.627116
